## Imports and declerations

In [28]:
import pickle
import os
import string
import re
import pprint
from IPython.core.interactiveshell import InteractiveShell
import numpy as np
import boolean
import pyparsing
from pyparsing import Word, alphas, oneOf, operatorPrecedence, opAssoc
InteractiveShell.ast_node_interactivity = "all"



os.getcwd()

'C:\\Users\\hassa\\Desktop\\Development\\Uni Projects\\Information Retrieval\\A1'

## Raw Vocabulary Storage:

In [61]:
from nltk.stem import PorterStemmer

In [51]:

# Break words like Veterans.Before, West.In amendment.Change

def split_words(vocabl):
    new_vocab = set()
    for word in vocabl:
        if re.search('^[a-zA-Z]+[.][a-zA-Z]+$',word) is not None:
            print(re.search('^[a-zA-Z]+[.][a-zA-Z]+$',word))
            w1, w2 = word.split('.')
#             print(w1)
# #             print(w2)
            new_vocab.add(w1)
            new_vocab.add(w2)
        elif re.search('^[a-zA-Z]+[?][a-zA-Z]+$',word) is not None:
# #             print(re.search('^[a-zA-Z]+[.][a-zA-Z]+$',word))
            w1, w2 = word.split('?')
# #             print(w1)
# #             print(w2)
            new_vocab.add(w1)
            new_vocab.add(w2)
        elif re.search('^[a-zA-Z]+[,][a-zA-Z]+$',word) is not None:
# #             print(re.search('^[a-zA-Z]+[.][a-zA-Z]+$',word))
            w1, w2 = word.split(',')
# #             print(w1)
# #             print(w2)
            new_vocab.add(w1)
            new_vocab.add(w2)
        else:
            new_vocab.add(word)
    return new_vocab

# Remove Punctuation
def remove_punctuation(word):
    return word.translate(word.maketrans('','',string.punctuation))


In [62]:
vocab = set()
doc_contents = []
printable = set(string.printable) 
# Printable characters are
# 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
# !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c


ps = PorterStemmer()

stop_words = set()
with open('Stopword-List.txt', 'r') as stop_word_file:
    lines = stop_word_file.readlines()
    for line in lines:
        stop_words.add(line.split('\n')[0])
    stop_words.remove('')

for file_number in range(0, 56):
    with open(f'data/Trump Speechs/speech_{file_number}.txt', 'r') as file1:
        lines = file1.readlines()
#         print(f'File Number : speech_{file_number}.txt' )
#         print(lines[0])
        for line in lines:
            doc_set = set()
            # split words at . , whitespace ? ! : ;
            for word in re.split('[.\s,?!:;-]', line):
                
                
                # Case Folding
                word = word.lower()
                
                # Filter non-ASCII characters
                word = ''.join(filter(lambda x: x in printable, word))
                
                # Remove Punctuations
                word = remove_punctuation(word)
                
                if re.match('\d+[A-Za-z]+',word):
                    word = re.split('\d+',word)[1]
                if re.match('[A-Za-z]+\d+',word):
                    word = re.split('\d+',word)[0]
                
                if len(word) == 0 or len(word) == 1 or word == '' or word == ' ':
                    continue
                if word in stop_words:
                    continue
                         
                word = ps.stem(word)
                    
                vocab.add(word)
                
                doc_set.add(word)
        
        doc_contents.append(doc_set)

In [63]:
print('Total Vocabulary Size ')
print(len(vocab))
print('Total Number of Documents ')
print(len(doc_contents))
print(doc_contents[17])

Total Vocabulary Size 
4850
Total Number of Documents 
56
{'refuge', 'civilian', 'averag', 'said', 'seem', 'unpaid', 'societi', 'ago', 'korea', 'sponsor', 'terrorist', 'shrink', 'week', 'waterway', 'near', '000', 'wasnt', 'worst', 'abil', 'enforc', 'racial', 'hillari', 'tax', 'abl', 'signific', 'count', '31', 'reagan', 'execut', 'build', 'warfight', 'immedi', 'so', 'million', 'onli', 'same', 'sometim', 'spend', 'action', 'review', 'common', 'speech', 'marin', 'ship', 'democraci', 'sequest', 'tomorrow', 'guid', 'defend', 'fast', 'unleash', 'didnt', 'medic', 'proudli', 'economi', 'benefit', 'run', 'direct', 'not', 'thing', 'divis', 'dead', 'get', 'rule', 'caught', 'with', 'terror', 'women', 'recommend', 'technolog', 'fbi', 'non', 'readi', 'largest', 'how', 'germani', 'were', 'against', 'exampl', 'expir', 'barack', 'look', 'level', 'arabia', 'support', 'potenti', 'return', 'libya', 'sound', 'that', 'depart', 'capabl', 'radic', 'down', 'disarray', 'substanti', 'be', 'command', 'note', 'dan

# Boolean Model:

In [64]:
# print(sorted(list(vocab)))
# for index,doc in enumerate(doc_contents):
#     print('Vocab size of doc' + str(index))
#     print(len(doc))

vocab_list = sorted(list(vocab))

term_doc_matrix_np = np.zeros((len(vocab), len(doc_contents)))

for word_index, word in enumerate(vocab_list):
    word_row = []
    for doc_index, doc in enumerate(doc_contents):
        if word in doc:
            term_doc_matrix_np[word_index, doc_index] = 1
        else:
            term_doc_matrix_np[word_index, doc_index] = 0
            
print(term_doc_matrix_np)
    

[[1. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [23]:

with open('pickled/vocab.p', 'ab') as vocab_file:
    pickle.dump(vocab, vocab_file)

In [24]:
with open('pickled/vocab.p', 'rb') as vocab_file:
    vocabf = pickle.load(vocab_file)

In [65]:
query = input('Enter your query : ')
query = ps.stem(query)
query_actions = []
query_wanted = []
if query in vocab:
    term_index = vocab_list.index(query)
    term_row = term_doc_matrix_np[term_index]
    print(term_row)
    doc_ids = np.argwhere(term_row == 1)
    print(doc_ids)

else:
    print(f'{query} not present in vocabulary')


Enter your query : running
[1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1.
 0. 0. 1. 1. 1. 1. 0. 0.]
[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [16]
 [17]
 [18]
 [19]
 [20]
 [21]
 [22]
 [24]
 [25]
 [26]
 [27]
 [28]
 [30]
 [32]
 [33]
 [34]
 [35]
 [36]
 [37]
 [39]
 [40]
 [41]
 [44]
 [45]
 [46]
 [47]
 [50]
 [51]
 [52]
 [53]]


In [66]:
test = '000dollar'
x = re.match('\d+[A-Za-z]+',test)
print(x)
x = re.split('\d+',test)
print(x)
# For matching queries like
# not hammer or pakistan
# (magnum or not hammer) or not (polish and pakistan)
x = re.match('(not)?\s*(\w+|(\((not)?\s*(\w+)\s+(and|or)\s+(not)?\s*(\w+)\)))\s+(or|and)\s+(not)?\s*(\w+|(\((not)?\s*(\w+)\s+(and|or)\s+(not)?\s*(\w+)\)))')



<re.Match object; span=(0, 9), match='000dollar'>
['', 'dollar']


In [66]:
ans = {'0', '1', '10', '11', '12', '16', '17', '18', '19', '2', '20', '21', '22', '24', '25', '26', '27', '28', '3', '30', '32', '33', '34', '35', '36', '37', '39', '4', '40', '41', '44', '45', '46', '47', '5', '50', '51', '52', '53', '6', '8', '9'}
ans2 =set([str(x[0]) for x in doc_ids])

In [67]:
print(len(ans))
print((ans2))
print(ans.difference(ans2))

42
{'22', '1', '53', '25', '37', '8', '24', '10', '0', '39', '50', '17', '35', '40', '21', '33', '27', '52', '3', '36', '20', '16', '26', '45', '9', '30', '2', '5', '12', '19', '4', '32', '28', '46', '18', '47', '41', '34', '11', '51', '6', '44'}
set()


# References :

http://www.pyregex.com/
http://cs231n.github.io/python-numpy-tutorial/

https://www.online-utility.org/text/analyzer.jsp

https://stackoverflow.com/questions/2118261/parse-boolean-arithmetic-including-parentheses-with-regex

https://regex101.com/r/M8z3U4/1

https://iq.opengenus.org/porter-stemmer/

In [85]:
query_actions = []
query_wanted = []
term_index = vocab_list.index(ps.stem('actions'))
term_row_actions = term_doc_matrix_np[term_index]

query_actions = np.argwhere(term_row_actions == 1)
term_index = vocab_list.index(ps.stem('wanted'))
term_row_wanted = term_doc_matrix_np[term_index]

query_wanted = np.argwhere(term_row_wanted == 1)

and_query = np.array([1 if x == 1 and y == 1 else 0 for x,y in zip(term_row_actions, term_row_wanted)])
and_doc_ids = np.argwhere(and_query == 1)
print(and_query)
print(and_doc_ids)

ans = {'37', '3', '19', '1', '9', '40', '51', '16', '15', '12', '31', '41', '39', '0', '53', '26', '29', '17', '24', '54', '7', '2', '5', '28', '42'}
ans2 =set([str(x[0]) for x in and_doc_ids])

print(len(ans))
print((ans2))
print(ans.difference(ans2))

[1 1 1 1 0 1 0 1 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 0
 1 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0 1 1 0]
[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 5]
 [ 7]
 [ 9]
 [12]
 [15]
 [16]
 [17]
 [19]
 [24]
 [26]
 [28]
 [29]
 [31]
 [37]
 [39]
 [40]
 [41]
 [42]
 [51]
 [53]
 [54]]
25
{'1', '53', '37', '24', '0', '39', '17', '54', '7', '40', '3', '16', '26', '31', '9', '42', '2', '5', '15', '12', '19', '28', '41', '51', '29'}
set()


TypeError: 'set' object is not callable

In [88]:
inp = "pakistan AND Running"
re.split('[\s\(\)]', inp)

['pakistan', 'AND', 'Running']

In [86]:


# Tokenizer Test
line = 'the Institute for Energy Research cites a "short-run" figure of as much as $36'
for word in re.split('[.\s,?!:;-]', line):            
    print(word)
    # Case Folding
    word = word.lower()
    print(word)
    # Filter non-ASCII characters
    word = ''.join(filter(lambda x: x in printable, word))
    print(word)
    # Remove Punctuations
    word = remove_punctuation(word)
    print(word)
    if re.match('\d+[A-Za-z]+',word):
        word = re.split('\d+',word)[1]
    if re.match('[A-Za-z]+\d+',word):
        word = re.split('\d+',word)[0]
    print(word)
    if len(word) == 0 or len(word) == 1 or word == '' or word == ' ':
        continue
    if word in stop_words:
        continue
    print(word)
    word = ps.stem(word)
    print(word)

the
the
the
the
the
Institute
institute
institute
institute
institute
institute
institut
for
for
for
for
for
Energy
energy
energy
energy
energy
energy
energi
Research
research
research
research
research
research
research
cites
cites
cites
cites
cites
cites
cite
a
a
a
a
a
"short
"short
"short
short
short
short
short
run"
run"
run"
run
run
run
run
figure
figure
figure
figure
figure
figure
figur
of
of
of
of
of
of
of
as
as
as
as
as
much
much
much
much
much
much
much
as
as
as
as
as
$36
$36
$36
36
36
36
36


In [114]:
from collections import deque 

line = 'united OR plane'
boperators = ['and', 'or']
uoperators = ['not']
  
stack = deque() 

def clean_word(word):
    # Case Folding
    word = word.lower()
     # Filter non-ASCII characters
    word = ''.join(filter(lambda x: x in printable, word))
#     print(word)
    # Remove Punctuations
    if word != '(' and word != ')':
        word = remove_punctuation(word)
#     print(word)
    if re.match('\d+[A-Za-z]+',word):
        word = re.split('\d+',word)[1]
    if re.match('[A-Za-z]+\d+',word):
        word = re.split('\d+',word)[0]
#     print(word)
    word = ps.stem(word)
#     print(word)
    return word
query = iter(re.split('[.\s,?!:;-]', line))

for word in query:  
    word = clean_word(word)
    print(word)
    if word not in boperators:
        if word not in vocab_list:
            print(f'{word} is not in vocabulary of index')
            break
        term_index = vocab_list.index(word)
        term_row = term_doc_matrix_np[term_index]
        
        stack.append({'state':True,'data':term_row})
    elif word in boperators:
        next_word = next(query)
        print(next_word)
        
        query1 =  stack.pop()
        query2 = clean_word(next_word)
        term_row1 = []
        if query1['state'] == False:
            term_index1 = vocab_list.index(query1['data'])
            term_row1 = term_doc_matrix_np[term_index1]
        
        else:
            term_row1 = query1['data']
        
        print(term_row1)
        term_index2 = vocab_list.index(query2)
        term_row2 = term_doc_matrix_np[term_index2]
        print(term_row2)
        if word == 'and':
            and_query = np.array([1 if x == 1 and y == 1 else 0 for x,y in zip(term_row1, term_row2)])
            print(and_query)
            and_doc_ids = np.argwhere(and_query == 1)
            query_ans =  set([x[0] for x in and_doc_ids])
            print(query_ans)
        elif word == 'or':
            and_query = np.array([1 if x == 1 or y == 1 else 0 for x,y in zip(term_row1, term_row2)])
            print(and_query)
            and_doc_ids = np.argwhere(and_query == 1)
            query_ans =  set([x[0] for x in and_doc_ids])
            print(query_ans)
    

unit
or
plane
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 1.]
[1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0.
 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0.]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1]
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55}


In [121]:
ansq = {'31', '28', '50', '46', '37', '30', '54', '10', '18', '7', '1', '17', '41', '49', '6', '34', '36', '11', '45', '29', '26', '52', '13', '21', '24', '16', '25', '32', '33', '4', '44', '22', '8', '19', '40', '20', '38', '48', '0', '47', '27', '51', '43', '2', '35', '39', '9', '3', '5', '12', '55'}
ansi = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55}
ansq2 = set([str(x) for x in ansi])
print(ansq2)
ansq2.difference(ansq)

{'22', '1', '25', '37', '8', '24', '10', '38', '0', '39', '50', '17', '13', '54', '7', '35', '40', '21', '33', '27', '52', '3', '36', '20', '16', '26', '31', '45', '9', '30', '49', '2', '5', '12', '19', '4', '55', '32', '28', '43', '44', '46', '18', '48', '47', '41', '34', '11', '51', '6', '29'}


set()